In [1]:
import pandas as pd
import pandas_datareader.data as web

In [2]:
import pandas_datareader as pdreader
pdreader.famafrench.get_available_datasets()

['F-F_Research_Data_Factors',
 'F-F_Research_Data_Factors_weekly',
 'F-F_Research_Data_Factors_daily',
 'F-F_Research_Data_5_Factors_2x3',
 'F-F_Research_Data_5_Factors_2x3_daily',
 'Portfolios_Formed_on_ME',
 'Portfolios_Formed_on_ME_Wout_Div',
 'Portfolios_Formed_on_ME_Daily',
 'Portfolios_Formed_on_BE-ME',
 'Portfolios_Formed_on_BE-ME_Wout_Div',
 'Portfolios_Formed_on_BE-ME_Daily',
 'Portfolios_Formed_on_OP',
 'Portfolios_Formed_on_OP_Wout_Div',
 'Portfolios_Formed_on_OP_Daily',
 'Portfolios_Formed_on_INV',
 'Portfolios_Formed_on_INV_Wout_Div',
 'Portfolios_Formed_on_INV_Daily',
 '6_Portfolios_2x3',
 '6_Portfolios_2x3_Wout_Div',
 '6_Portfolios_2x3_weekly',
 '6_Portfolios_2x3_daily',
 '25_Portfolios_5x5',
 '25_Portfolios_5x5_Wout_Div',
 '25_Portfolios_5x5_Daily',
 '100_Portfolios_10x10',
 '100_Portfolios_10x10_Wout_Div',
 '100_Portfolios_10x10_Daily',
 '6_Portfolios_ME_OP_2x3',
 '6_Portfolios_ME_OP_2x3_Wout_Div',
 '6_Portfolios_ME_OP_2x3_daily',
 '25_Portfolios_ME_OP_5x5',
 '25_Portf

In [3]:
startdate = '19800101'
enddate = '20210930'

In [4]:
info = pd.DataFrame(columns=['Name', 'Unit', 'Construction', 'Description'])
info.loc['MKT'] = ['Market', 'Excess Return', 'Market-cap-weighted', 'US Equities']
info.loc['SMB'] = ['Size', 'Excess Return', 'Small Minus Big', 'Long small stocks and short big stocks']
info.loc['HML'] = ['Value', 'Excess Return', 'High Minus Low', 'Long value (high book-to-market) stocks and short growth (low book-to-market) stocks']
info.loc['RMW'] = ['Profitability','Excess Return', 'Robust Minus Weak', 'Long profitability (income statement) and short low profitability']
info.loc['CMA'] = ['Investment','Excess Return', 'Conservative Minus Agressive', 'Long stocks with low (conservative) investment (balance sheet) and short stocks with high (agressive) investment.']
info.loc['UMD'] = ['Momentum','Excess Return','Up Minus Down', 'Long stocks that have high recent returns and short stocks with low recent returns']
info.loc['RF'] = ['Risk-free rate', 'Total Return', 'Tbills', '']
info

,Name,Unit,Construction,Description
MKT,Market,Excess Return,Market-cap-weighted,US Equities
SMB,Size,Excess Return,Small Minus Big,Long small stocks and short big stocks
HML,Value,Excess Return,High Minus Low,Long value (high book-to-market) stocks and sh...
RMW,Profitability,Excess Return,Robust Minus Weak,Long profitability (income statement) and shor...
CMA,Investment,Excess Return,Conservative Minus Agressive,Long stocks with low (conservative) investment...
UMD,Momentum,Excess Return,Up Minus Down,Long stocks that have high recent returns and ...
RF,Risk-free rate,Total Return,Tbills,


In [5]:
rawdata = web.DataReader('F-F_Research_Data_5_Factors_2x3', data_source='famafrench',start=startdate,end=enddate)
facs = rawdata[0] / 100
# convert the yyyy-mm to timestamp object. default is to put first of month, but should be end of month
facs = facs.to_timestamp().resample('M').last()
facs.rename(columns={'Mkt-RF':'MKT'},inplace=True)

rawdata = web.DataReader('F-F_Momentum_Factor', data_source='famafrench',start=startdate,end=enddate)
umd = rawdata[0] / 100
# convert the yyyy-mm to timestamp object. default is to put first of month, but should be end of month
umd = umd.to_timestamp().resample('M').last()
umd.columns = ['UMD']

rf = facs[['RF']]
facs = facs.join(umd).drop(columns=['RF'])
facs

,MKT,SMB,HML,RMW,CMA,UMD
Date,,,,,,
1980-01-31,0.0551,0.0182,0.0175,-0.0169,0.0171,0.0755
1980-02-29,-0.0122,-0.0158,0.0061,0.0007,0.0270,0.0788
1980-03-31,-0.1290,-0.0692,-0.0101,0.0145,-0.0116,-0.0955
1980-04-30,0.0397,0.0105,0.0108,-0.0206,0.0027,-0.0043
1980-05-31,0.0526,0.0210,0.0038,0.0032,-0.0032,-0.0112
...,...,...,...,...,...,...
2021-05-31,0.0029,0.0119,0.0704,0.0234,0.0310,0.0088
2021-06-30,0.0275,-0.0025,-0.0776,-0.0214,-0.0099,0.0224
2021-07-31,0.0127,-0.0456,-0.0175,0.0536,-0.0055,-0.0229


In [6]:
rawdata = web.DataReader('49_Industry_Portfolios', data_source='famafrench',start=startdate, end=enddate)
rets = rawdata[0] / 100
# convert the yyyy-mm to timestamp object. default is to put first of month, but should be end of month
rets = rets.to_timestamp().resample('M').last()
retsx = rets.subtract(rf['RF'],axis=0)
retsx

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1980-01-31,-0.0050,0.0283,0.0084,0.1024,-0.0143,0.0999,0.0354,0.0352,0.0048,0.0032,...,0.0159,0.0876,0.0463,-0.0119,0.0458,-0.0279,0.0258,0.0751,0.0299,0.0663
1980-02-29,0.0111,-0.0610,-0.0966,-0.0319,-0.0569,-0.0314,-0.0527,-0.0788,-0.0556,-0.0140,...,-0.0079,-0.0535,-0.0339,-0.0636,-0.0638,-0.0855,-0.0960,-0.0314,-0.0275,-0.0266
1980-03-31,-0.2244,-0.1116,-0.0167,-0.1464,-0.0192,-0.1281,-0.0817,-0.1278,-0.0565,-0.0664,...,-0.0821,-0.1511,-0.1106,-0.0913,-0.1443,-0.0563,-0.0883,-0.2441,-0.1243,-0.1733
1980-04-30,0.0451,0.0766,0.0232,0.0305,0.0831,-0.0521,0.0775,0.0182,0.0304,0.0113,...,0.0419,-0.0097,-0.0300,0.0341,0.0522,0.0729,0.0532,0.0997,0.0448,0.0771
1980-05-31,0.0637,0.0792,0.0457,0.0895,0.0814,0.0512,0.0324,0.0876,0.0560,0.0064,...,0.0565,0.1060,0.1147,0.0872,0.1127,0.0577,0.0557,0.1040,0.0839,0.0681
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-31,-0.0612,0.0394,0.0199,0.0199,0.0204,0.0294,-0.0086,0.0290,0.0029,-0.0057,...,-0.0353,0.0214,0.0107,-0.0282,-0.0125,0.0273,0.0291,0.0128,0.0455,0.0488
2021-06-30,-0.0235,-0.0306,-0.0145,-0.0098,0.0199,0.1201,0.0156,-0.0023,-0.0026,0.0882,...,-0.0359,-0.0429,-0.0182,0.0373,-0.0077,-0.0290,-0.0338,-0.0099,0.0163,-0.0366
2021-07-31,-0.0306,-0.0229,0.0453,0.0109,0.0096,-0.0307,-0.0745,-0.0255,0.0391,0.0454,...,-0.0142,-0.0490,0.0238,-0.0079,0.0450,-0.0025,0.0011,0.0426,0.0171,-0.0012


In [7]:
with pd.ExcelWriter('factor_pricing_data.xlsx') as writer:  
    info.to_excel(writer, sheet_name= 'descriptions')
    facs.to_excel(writer, sheet_name='factors (excess returns)')
    retsx.to_excel(writer, sheet_name='portfolios (excess returns)')
    rf.to_excel(writer, sheet_name='risk-free rate')